In [ ]:
webcam with ss

In [ ]:
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import mediapipe as mp
import os
from datetime import datetime

# MediaPipe Hand and Pose Modules
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)
pose = mp_pose.Pose(min_detection_confidence=0.7)

# Define the violence detection model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification output
])

# Preprocessing function: resize and normalize the frame
def preprocess_frame(frame):
    frame_resized = cv2.resize(frame, (224, 224))
    frame_normalized = frame_resized / 255.0
    return frame_normalized

# Function to detect humans using HOG descriptor
def detect_humans(frame):
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect humans in the frame
    humans, _ = hog.detectMultiScale(gray_frame, winStride=(8, 8), padding=(8, 8), scale=1.05)
    
    for (x, y, w, h) in humans:
        # Draw bounding box around each detected human
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Object detection (simplified using Haar cascades for face detection as an example)
def detect_objects(frame):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect objects (faces in this example)
    faces = face_cascade.detectMultiScale(gray_frame, 1.1, 4)
    
    for (x, y, w, h) in faces:
        # Draw bounding box around detected objects (faces)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

# Detect specific actions such as boxing, punching, or slapping using MediaPipe and velocity/position analysis
def detect_specific_actions(frame, previous_landmarks):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Detect poses
    results_pose = pose.process(frame_rgb)
    action_type = "None"
    body_action_detected = False

    if results_pose.pose_landmarks:
        landmarks = results_pose.pose_landmarks.landmark
        
        # Get important landmarks for detecting actions
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
        right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

        # Calculate the velocity of wrists to check for fast hand movements
        if previous_landmarks:
            left_wrist_velocity = abs(left_wrist.x - previous_landmarks[0]) + abs(left_wrist.y - previous_landmarks[1])
            right_wrist_velocity = abs(right_wrist.x - previous_landmarks[2]) + abs(right_wrist.y - previous_landmarks[3])

            # Set a threshold for detecting fast hand movements like slapping or punching
            velocity_threshold = 0.1

            # Detect boxing or slapping based on velocity and hand position
            if left_wrist_velocity > velocity_threshold or right_wrist_velocity > velocity_threshold:
                if abs(left_wrist.y - left_elbow.y) < 0.15 or abs(right_wrist.y - right_elbow.y) < 0.15:
                    action_type = "Boxing/Punching"
                    body_action_detected = True
                elif abs(left_wrist.x - left_shoulder.x) > 0.2 or abs(right_wrist.x - right_shoulder.x) > 0.2:
                    action_type = "Slapping"
                    body_action_detected = True
        
        # Save current wrist positions for the next frame
        previous_landmarks = [left_wrist.x, left_wrist.y, right_wrist.x, right_wrist.y]
    else:
        previous_landmarks = None

    return action_type, body_action_detected, previous_landmarks

# Capture and save screenshots when violence is detected
def capture_screenshot(frame):
    # Create a unique filename using the timestamp
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    filename = f"violence_detected_{timestamp}.png"
    
    # Define the folder where screenshots will be saved
    screenshot_folder = 'screenshots'
    
    # Create the folder if it doesn't exist
    if not os.path.exists(screenshot_folder):
        os.makedirs(screenshot_folder)
    
    # Save the screenshot in the specified folder
    screenshot_path = os.path.join(screenshot_folder, filename)
    cv2.imwrite(screenshot_path, frame)
    print(f"Screenshot captured and saved at {screenshot_path}")

# Violence detection function with action recognition, bounding boxes, and screenshots
def detect_violence_in_webcam(model):
    cap = cv2.VideoCapture(0)  # Open the laptop webcam
    violence_detected = False
    alert_message = "PoliceAlert: Negative"
    violence_text = "Violence Detected: No"
    action_detected_text = "Action Detected: None"
    previous_landmarks = None  # Track previous wrist positions for action detection

    print("Starting webcam feed for real-time violence detection...")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame from webcam.")
            break

        # Preprocess the frame for violence detection
        processed_frame = preprocess_frame(frame)
        
        # Predict violence
        prediction = model.predict(np.expand_dims(processed_frame, axis=0))
        prediction_score = prediction[0][0]  # Get the actual prediction score
        
        # Detect specific actions like boxing, punching, or slapping
        action_type, action_detected, previous_landmarks = detect_specific_actions(frame, previous_landmarks)

        # Update violence and action detection status
        if prediction_score > 0.5 and action_detected:
            violence_detected = True
            alert_message = "PoliceAlert: Positive"
            violence_text = "Violence Detected: Yes"
            action_detected_text = f"Action Detected: {action_type}"
            
            # Capture and save the screenshot when violence is detected
            capture_screenshot(frame)
        else:
            alert_message = "PoliceAlert: Negative"
            violence_text = "Violence Detected: No"
            action_detected_text = "Action Detected: None"
        
        # Detect humans in the frame and draw bounding boxes
        detect_humans(frame)

        # Detect objects (e.g., faces in this example) in the frame
        detect_objects(frame)

        # Display the violence and action detection status at the top of the video
        cv2.putText(frame, violence_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.putText(frame, alert_message, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(frame, action_detected_text, (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the video frame by frame
        cv2.imshow('Webcam Feed - Real-Time Detection', frame)

        # Add a small delay and break if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()  # Automatically close the window after webcam feed ends

# Run the violence detection using webcam
detect_violence_in_webcam(model)


I0000 00:00:1745559560.413189  557293 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
I0000 00:00:1745559560.418654  557293 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1745559560.437601  557473 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745559560.443908  557473 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 0

Starting webcam feed for real-time violence detection...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


W0000 00:00:1745559562.449578  557486 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
/opt/anaconda3/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Screenshot captured and saved at screenshots/violence_detected_2025-04-25_11-09-26.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
S

In [ ]:
webcam with ss and beep sound

In [ ]:
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import mediapipe as mp
import os
from datetime import datetime
from playsound import playsound
import time

# Paths for YOLO model files
weights_path = '/Users/samarthmaheshsalvi/Downloads/SEM 7/FINAL YEAR PROJECT copy/yolov3.weights'
config_path = '/Users/samarthmaheshsalvi/Downloads/SEM 7/FINAL YEAR PROJECT copy/yolov3.cfg'
labels_path = '/Users/samarthmaheshsalvi/Downloads/SEM 7/FINAL YEAR PROJECT copy/coco.names'

# Load YOLO model
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Load class names (COCO dataset labels)
with open(labels_path, 'r') as f:
    classes = f.read().strip().split('\n')

# MediaPipe for Pose detection
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.7)

# Violence detection CNN model (simplified)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification output
])

# Preprocess frame for CNN model
def preprocess_frame(frame):
    frame_resized = cv2.resize(frame, (224, 224))
    frame_normalized = frame_resized / 255.0
    return frame_normalized

# Detect specific actions like slapping and boxing using MediaPipe Pose
def detect_actions(frame, previous_landmarks):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_pose = pose.process(frame_rgb)
    action_type = "None"
    action_detected = False

    if results_pose.pose_landmarks:
        landmarks = results_pose.pose_landmarks.landmark
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
        right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]

        if previous_landmarks:
            # Calculate velocities of wrists
            left_wrist_velocity = abs(left_wrist.x - previous_landmarks[0]) + abs(left_wrist.y - previous_landmarks[1])
            right_wrist_velocity = abs(right_wrist.x - previous_landmarks[2]) + abs(right_wrist.y - previous_landmarks[3])
            velocity_threshold = 0.003  # Lower threshold for more sensitivity

            # Detect if there is quick movement of wrists near elbows (indicating boxing or slapping)
            if left_wrist_velocity > velocity_threshold or right_wrist_velocity > velocity_threshold:
                if abs(left_wrist.y - left_elbow.y) < 0.15 or abs(right_wrist.y - right_elbow.y) < 0.15:
                    if abs(left_wrist.y - left_elbow.y) < 0.05 or abs(right_wrist.y - right_elbow.y) < 0.05:
                        action_type = "Slapping"
                    else:
                        action_type = "Boxing"
                    action_detected = True

        previous_landmarks = [left_wrist.x, left_wrist.y, right_wrist.x, right_wrist.y]
    else:
        previous_landmarks = None

    return action_type, action_detected, previous_landmarks

# Play beep sound for 2 seconds
def play_alert_sound():
    sound_path = '/Users/samarthmaheshsalvi/Downloads/SEM 7/FINAL YEAR PROJECT copy/alert_beep.mp3'
    playsound(sound_path)

# Capture and save screenshots
def capture_screenshot(frame):
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    screenshot_folder = '/Users/samarthmaheshsalvi/Downloads/SEM 7/FINAL YEAR PROJECT copy/screenshots'
    
    if not os.path.exists(screenshot_folder):
        os.makedirs(screenshot_folder)
    
    screenshot_path = os.path.join(screenshot_folder, f"violence_detected_{timestamp}.png")
    cv2.imwrite(screenshot_path, frame)
    print(f"Screenshot saved at {screenshot_path}")

# Main function for real-time violence detection using the webcam
def detect_violence_in_webcam(model):
    cap = cv2.VideoCapture(0)
    previous_landmarks = None
    red_text_end_time = 0
    violence_detected = False
    alert_message = "AuthorityAlert: Negative"
    text_color = (0, 255, 0)  # Default green color for no violence

    print("Starting webcam feed...")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            break

        processed_frame = preprocess_frame(frame)
        prediction = model.predict(np.expand_dims(processed_frame, axis=0))
        prediction_score = prediction[0][0]

        action_type, action_detected, previous_landmarks = detect_actions(frame, previous_landmarks)
        current_time = time.time()

        if action_detected and prediction_score > 0.5:
            violence_detected = True
            alert_message = "AuthorityAlert: Positive"
            text_color = (0, 0, 255)  # Red color for violence
            capture_screenshot(frame)
            play_alert_sound()
            red_text_end_time = current_time + 2  # Show red text for 2 seconds

        if current_time < red_text_end_time:
            # Display red text during the alert period
            cv2.putText(frame, "Violence Detected: Yes", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2)
            cv2.putText(frame, f"Action Detected: {action_type}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2)
            cv2.putText(frame, alert_message, (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2)
        else:
            violence_detected = False
            alert_message = "AuthorityAlert: Negative"
            text_color = (0, 255, 0)  # Green color for no violence
            cv2.putText(frame, "Violence Detected: No", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2)
            cv2.putText(frame, "Action Detected: None", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2)
            cv2.putText(frame, alert_message, (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2)

        cv2.imshow('Webcam - Violence Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Run the detection
detect_violence_in_webcam(model)


I0000 00:00:1745575116.159729  619839 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1745575116.288767  651603 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745575116.442128  651603 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Starting webcam feed...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step


W0000 00:00:1745575117.583497  651602 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
/opt/anaconda3/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 

In [ ]:
webcam with button code

In [1]:
!pip install PyQt5

In [1]:
import sys
import cv2
import numpy as np
import os
from datetime import datetime
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QLabel, QLineEdit, QMainWindow, QMessageBox
from PyQt5.QtCore import QThread, pyqtSignal
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import mediapipe as mp

# Define violence detection model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification output
])

# Preprocessing function for the video frame
def preprocess_frame(frame):
    frame_resized = cv2.resize(frame, (224, 224))
    frame_normalized = frame_resized / 255.0
    return frame_normalized

# Screenshot function
def capture_screenshot(frame):
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    filename = f"violence_detected_{timestamp}.png"
    screenshot_folder = 'screenshots'
    
    if not os.path.exists(screenshot_folder):
        os.makedirs(screenshot_folder)
    
    screenshot_path = os.path.join(screenshot_folder, filename)
    cv2.imwrite(screenshot_path, frame)
    print(f"Screenshot captured and saved at {screenshot_path}")

# Violence Detection Thread
class DetectionThread(QThread):
    change_pixmap_signal = pyqtSignal(np.ndarray)
    
    def __init__(self, source=0):
        super().__init__()
        self.source = source

    def run(self):
        cap = cv2.VideoCapture(self.source)
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            processed_frame = preprocess_frame(frame)
            prediction = model.predict(np.expand_dims(processed_frame, axis=0))
            prediction_score = prediction[0][0]

            if prediction_score > 0.5:
                capture_screenshot(frame)
                cv2.putText(frame, "Violence Detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                cv2.putText(frame, "No Violence", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            self.change_pixmap_signal.emit(frame)

        cap.release()

# Main Application Window
class ViolenceDetectionApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Violence Detection System")
        self.setGeometry(100, 100, 800, 600)
        self.current_user = None

        # Login Page UI
        self.login_label = QLabel("Login Page", self)
        self.login_label.setGeometry(350, 50, 100, 50)

        self.username_input = QLineEdit(self)
        self.username_input.setPlaceholderText("Enter Username")
        self.username_input.setGeometry(300, 150, 200, 40)

        self.password_input = QLineEdit(self)
        self.password_input.setEchoMode(QLineEdit.Password)
        self.password_input.setPlaceholderText("Enter Password")
        self.password_input.setGeometry(300, 200, 200, 40)

        self.login_button = QPushButton("Login", self)
        self.login_button.setGeometry(350, 260, 100, 40)
        self.login_button.clicked.connect(self.check_login)

    def check_login(self):
        username = self.username_input.text()
        password = self.password_input.text()

        if username == "admin" and password == "password":
            self.current_user = username
            self.show_main_menu()
        else:
            QMessageBox.warning(self, "Error", "Incorrect login credentials!")

    def show_main_menu(self):
        # Clear login page
        self.login_label.hide()
        self.username_input.hide()
        self.password_input.hide()
        self.login_button.hide()

        # Main Menu
        self.start_detection_button = QPushButton("Start Detection", self)
        self.start_detection_button.setGeometry(250, 200, 300, 50)
        self.start_detection_button.clicked.connect(self.start_video_detection)

        self.real_time_detection_button = QPushButton("Start Real-Time Detection", self)
        self.real_time_detection_button.setGeometry(250, 300, 300, 50)
        self.real_time_detection_button.clicked.connect(self.start_real_time_detection)

    def start_video_detection(self):
        # File based detection code
        self.detection_thread = DetectionThread('path/to/your/video.mp4')  # Update path as needed
        self.detection_thread.change_pixmap_signal.connect(self.update_frame)
        self.detection_thread.start()

    def start_real_time_detection(self):
        # Real-time webcam detection code
        self.detection_thread = DetectionThread(0)  # 0 for webcam
        self.detection_thread.change_pixmap_signal.connect(self.update_frame)
        self.detection_thread.start()

    def update_frame(self, frame):
        # Update GUI with frames during detection
        # You can enhance this method to display frames in a QLabel
        pass

# Main function to run the app
def main():
    app = QApplication(sys.argv)
    window = ViolenceDetectionApp()
    window.show()
    sys.exit(app.exec_())

if __name__ == "__main__":
    main()


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-10-03 10:55:45.886 python[1601:41253] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-03 10:55:45.886 python[1601:41253] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


SystemExit: 0

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
